In [ ]:
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv
import os
import random
import matplotlib.pyplot as plt
import h_louvain_BO as hlbo
import pickle


In [ ]:
def build_habcd():
    cmd = julia + ' -n ' + str(nodes)
    cmd = cmd + ' -d ' + str(deg_exp) + ',' + str(deg_min) + ',' + str(deg_max)
    cmd = cmd + ' -c ' + str(com_exp) + ',' + str(com_min) + ',' + str(com_max)
    cmd = cmd + ' -x ' + str(xi)
    cmd = cmd + ' -q ' + str(edge_sizes[0])
    for i in np.arange(1,len(edge_sizes),1):
        cmd = cmd + ',' + str(edge_sizes[i])
    cmd = cmd + ' -w wcd.txt'
    cmd = cmd + ' -s ' + str(seed)
    cmd = cmd + ' -o hgwcd >& _temp'     
    return cmd

In [ ]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [ ]:
def get_AMI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ami = [d[str(i+1)] for i in range(len(HG.nodes))]
    return AMI(gt, A4ami)


In [ ]:
julia = '/Applications/Julia-1.7.app/Contents/Resources/julia/bin/julia --project /Users/francois/ABCD/ABCDHypergraphGenerator/utils/abcdh.jl'

nodes = 300
deg_min = 5
deg_max = 30
deg_exp = 2.5
com_min = 40
com_max = 60
com_exp = 1.5
edge_sizes = [0,0,0,0,1]
#edge_type = 'majority'

# ## FCT - experimental modularity function


In [ ]:
xi = 0.2
L = []
for rep in range(30):
    cmd = build_habcd()
    os.system(cmd)
    seed = rep
    random.seed(rep)

    with open("./hgwcd_he.txt","r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    with open("./hgwcd_assign.txt", 'r') as file:
        gt = [int(line) for line in file]
    gt_dict = {str(i+1):gt[i] for i in range(len(gt))}

    mc = max(gt)
    Nodes = [x for x in gt_dict.keys() if gt_dict[x]>=(mc-1)]
    for i in range(25):
        Edges.append(set(np.random.choice(Nodes,5,replace=False)))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    
    hL = hl.hLouvain(HG, hmod_type=expo, delta_it = 0.0001, delta_phase = 0.0001, random_seed = seed) 
    A_gt = [x for x in hmod.dict2part(gt_dict) if len(x)>0]
    qH_gt = hL.combined_modularity(A_gt, expo, 1)

    ## two-section
    G = hnx.two_section(HG)
    dct = hnx.part2dict(A_gt)
    for v in G.vs:
        v['gt'] = dct[v['name']]   
    qH_ts = -1
    for ts_rep in range(5):
        ml = G.community_multilevel(weights='weight').membership
        q = hL.combined_modularity(hnx.dict2part(dict(zip(G.vs['name'], ml))), expo, 0)
        if q > qH_ts:
            ami_ts = AMI(G.vs['gt'],ml)
            qH_ts = q 
    V = [ami_ts]
    for ex in [0,1,2,3,4,5,6,7,8]:
        def expo(d, c):
            return (c/d)**(ex/2) if c > d/2 else 0  
        hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
        hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
        df_bo = hBO.hLouvain_perform_BO()
        qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
        ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
        V.append(ami_bo)
    L.append(V)
    print(V)
    fn = 'test_5.pkl'
    with open(fn, 'wb') as fp:
        pickle.dump(L, fp)
